In [1]:
#http://datareview.info/article/sovremennyie-metodyi-analiza-tonalnosti-teksta/
import pandas
import numpy as np
positive_raw = pandas.read_csv('positive.csv', sep=';', header=None)
negative_raw = pandas.read_csv('negative.csv', sep=';', header=None)
print(type(positive_raw[[3,4]]))
import warnings
warnings.filterwarnings('ignore')

<class 'pandas.core.frame.DataFrame'>


In [2]:
all_raw = positive_raw.append(negative_raw)
all_raw = all_raw[[3, 4]]
all_raw.columns = ['text', 'isPositive']


In [2]:
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def get_normalized_form(word):
    return morph.parse(word)[0].normal_form

regex = re.compile(r"@[\w]+")
def proceed_string(input_str):
    input_str = input_str.replace('\n', ' ')
    input_str = input_str.replace('\\n', ' ')
    input_str = re.sub(r"@[\w]+:", "", input_str)
    input_str = re.sub(r"[^ЁёА-Яа-я\s]", " ", input_str)
    input_str = input_str.lower()
    words_arr = input_str.split()
    words_arr = [get_normalized_form(w) for w in words_arr]
    return words_arr

ModuleNotFoundError: No module named 'pymorphy2'

In [1]:
all_raw['text'] = all_raw['text'].map(proceed_string)

NameError: name 'all_raw' is not defined

In [5]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(all_raw['text'], all_raw['isPositive'], test_size = 0.2)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
n_dim = 200
from gensim.models.word2vec import Word2Vec
twits_w2v = Word2Vec(size=n_dim, min_count=5)
twits_w2v.build_vocab(x_train)
twits_w2v.train(x_train, total_examples=len(x_train), epochs=5)

(7289988, 9987315)

In [8]:
def word_arr_to_vec(arr, length):
    vec = np.zeros(length)
    count = 0
    for word in arr:
        try:
            vec += twits_w2v[word]
            count += 1.0
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec
            

In [9]:
from sklearn.preprocessing import scale
train_vecs = np.array([word_arr_to_vec(arr, 200) for arr in x_train])
train_vecs = scale(train_vecs)


In [10]:
train_vecs

array([[-1.42992090e-01, -1.25467266e+00,  4.29186582e-01, ...,
        -4.56731803e-01,  1.18987826e+00, -9.12156055e-01],
       [-1.81874605e-01, -6.73705269e-01,  8.95897185e-01, ...,
         8.54989827e-01,  4.08406610e-01,  8.13921492e-02],
       [ 1.59217758e+00,  9.69355688e-01,  4.17481647e-01, ...,
         7.44258599e-01, -4.48830863e-01,  6.74723104e-01],
       ...,
       [-4.65848808e-01, -1.92089091e-01,  8.14199162e-01, ...,
        -1.60599244e+00, -2.81689834e-01, -3.80773276e-01],
       [ 5.34540833e-01, -9.00238657e-01,  9.92915665e-05, ...,
         6.41597905e-01, -1.20730544e-01, -3.41209666e-01],
       [ 9.31829394e-01,  1.33014805e-01,  5.56071351e-01, ...,
        -5.09435601e-01, -1.36153644e+00,  2.23834933e-01]])

In [11]:
#twits_w2v.train(x_test, total_examples=len(x_test), epochs=5)
test_vecs = np.array([word_arr_to_vec(arr, 200) for arr in x_test])
test_vecs = scale(test_vecs)

In [12]:
from sklearn.linear_model import SGDClassifier

lr = SGDClassifier(loss='log', penalty='l1')
lr.fit(train_vecs, y_train)

print ('Test Accuracy: %.2f'%lr.score(test_vecs, y_test))

Test Accuracy: 0.64


In [13]:
twits_w2v.most_similar(positive = 'плохой')

[('страшный', 0.7219053506851196),
 ('странный', 0.7196283340454102),
 ('хороший', 0.6710162162780762),
 ('важный', 0.6643310785293579),
 ('ужасный', 0.6614466905593872),
 ('смешной', 0.6472550630569458),
 ('портить', 0.6304638981819153),
 ('умный', 0.6263653039932251),
 ('скучный', 0.6254901885986328),
 ('обычный', 0.622004508972168)]